In [ ]:
%matplotlib notebook


In [ ]:
%%capture
%run UnscentedKalman.ipynb


$$
\def\M#1{{[\mathbf{#1}]}}
\def\MM#1#2{{[\mathbf{#1}{#2}]}}
\def\E{\operatorname{E}}
\def\cov{\operatorname{cov}}
\def\T{^\mathsf{T}}
$$

# Augmented state
If we are considering nonlinear transformations, we might as well consider nonlinear process noise as well. Instead of in the form:

$$\begin{eqnarray*}
\vec{x}^-&=&f(\vec{x}_{i-1})+\vec{v} \\
\MM{P}{^-}&=&UT(f,\MM{P}{_{i-1}})+\M Q
\end{eqnarray*}$$

we *augment* the state vector with a process noise vector. 

$$\vec{x}_a=\begin{bmatrix}\vec{x} \\ - \\ \vec{v}\end{bmatrix}$$

We will need an augmented physics function $f_a$ as well, which accepts an augmented vector and uses the noise vector components to implement noise (not in general adding the noise). The return vector will be un-augmented, and therefore the covariance is compatible with the un-augmented state vector.

So if we have a state vector $\vec{x}$ with $n$ components, and noise vector $\vec{v}$ with $\ell$ components, we will end up with an augmented state vector $\vec{x}_a$ with $n+\ell$ components. We then generate sigma points with an augmented covariance -- often it is a block-diagonal covariance like this:

$$\MM{P}{_a}=\begin{bmatrix}\M P & \MM{0}{_{n \times \ell}} \\
                               \MM{0}{_{\ell \times n}} & \M{Q}\end{bmatrix}$$
                               
The off-diagonal blocks are the cross-covariance between the state and the process noise. It's fair to treat them as zero, meaning that there is no correlation between the state and the noise.

When we generate sigma points with the augmented covariance, we will end up with $2(n+\ell)+1$ sigma points, each of which is $n+\ell$ elements long. After running all of them through the augmented nonlinear transformation, we will end up with $2(n+\ell)+1$ points still, but now each vector is only $n$ elements long. When we calculate the updated covariance $\M P$, it will only be $n \times n$.

$$\begin{eqnarray*}
\vec{x}^-&=&f(x_{a,i-1}) \\
\MM{P}{^-}&=&UT(f_a,\MM{P}{_{a,i-1}})
\end{eqnarray*}$$

This particular problem is quite sensitive to the integrator and number of time substeps used. 

# Example 2 - Atmospheric Entry
Let's try another one -- the dreaded Atmospheric Entry problem that the original authors used as their test case.

The following problem is simple to describe but apparently is a difficult problem for the extended Kalman filter to handle. 

A spherical reentry vehicle (no lift) is entering the atmosphere of Earth at a relatively steep angle and orbital speed. The rotation of the Earth is neglected, and all motion happens in a plane. 

The atmosphere model is a simple exponential, and the ballistic coefficient not a function of mach number. The ballistic coefficient is subject to a random walk, as are both components of the velocity, but not either component of the position.

A radar station is tracking the vehicle, producing range and azimuth measurements at a high rate. 
## Equations of Motion
The state vector $\vec{x}$ to be estimated is the 2D position ($x_1$ and $x_2$), 2D velocity ($x_3$ and $x_4$), and ballistic coefficient of the vehicle ($x_5$). As stated above, the motion is subject to a random walk $\vec{v}$ on three components, the 2D velocity ($v_1$ and $v_2$) and ballistic coefficient ($v_3$). The equations of motion of the vehicle are then as follows:

$$\begin{eqnarray*}
\dot{x}_1&=&x_3\\
\dot{x}_2&=&x_4\\
\dot{x}_3&=&Dx_3+Gx_1+v_1\\
\dot{x}_4&=&Dx_4+Gx_2+v_2\\
\dot{x}_5&=&v_3
\end{eqnarray*}$$

where $D$ is the drag-related force term, $G$ is the gravity-related force term, $R$ is the distance from the reentry vehicle to the center of the Earth, $R_0$ is the radius of the Earth, $Gm_0$ is the gravitational parameter of the Earth, and $H_0$ is the density scale height of the atmosphere. These terms are evaluated as follows:

{{sidebar|Both UKF0.pfd and UKF1.pdf had an error here, $\beta_0$ should be positive}}
$$\begin{align}
R_0=&6374\mbox{ km}\\
Gm_0=&3.9860\times 10^5 \mbox{ km}^3/\mbox{s}^2\\
H_0=&13.406 \mbox{ km}\\
\beta_0=&0.59783 \\
R=&\sqrt{x_1^2+x_2^2}\\
V=&\sqrt{x_3^2+x_4^2}\\
\beta=&\beta_0 e^{x_5}\\
G=&-\frac{Gm_0}{R^3}\\
D=&-\beta V e^{\left\{\frac{R_0-R}{H_0}\right\}}\\
\end{align}$$

The $G$ term does in fact produce the centrally-directed inverse-square term we've grown to know and love, and the $D$ term does produce a drag opposite in direction to the velocity with magnitude proportional to the square of the speed.

The ballistic coefficient $\beta$ is calculated in this roundabout manner rather than directly estimated because the filter is not smart enough to constrain the ballistic coefficient to be positive. With this indirect method, any real number state vector component produces a valid positive ballistic coefficient.

In [ ]:
x_1=sympy.symbols("x_1")
x_2=sympy.symbols("x_2")
x_3=sympy.symbols("x_3")
x_4=sympy.symbols("x_4")
x_5=sympy.symbols("x_5")
#v_1=sympy.symbols("v_1")
#v_2=sympy.symbols("v_2")
#v_3=sympy.symbols("v_3")
R_0=6374
Gm_0=3.9860e5
H_0=13.406
beta_0=0.59783
R=sympy.sqrt(x_1**2+x_2**2)
V=sympy.sqrt(x_3**2+x_4**2)
beta=beta_0*sympy.exp(x_5)
G=-Gm_0/R**3
D=-beta*V*sympy.exp((R_0-R)/H_0)
Fx_1=x_3
Fx_2=x_4
Fx_3=D*x_3+G*x_1#+v_1
Fx_4=D*x_4+G*x_2#+v_2
Fx_5=0#v_3
display(sympy.simplify(Fx_3))
display(sympy.simplify(Fx_4))
(Fre,Phire)=fjac([x_1,x_2,x_3,x_4,x_5],[],[Fx_1,Fx_2,Fx_3,Fx_4,Fx_5])


## Measurement equations
The vehicle is tracked by a radar at $(x_r,y_r)$ which produces range and azimuth measurements. The measurements are determined from the state vector as follows:

$$\begin{align}
r_r=&\sqrt{(x_1-x_r)^2+(x_2-y_r)^2}+w_1\\
\theta=&\tan^{-1}\left(\frac{x_2-y_r}{x_1-x_r}\right)+w_2
\end{align}$$

{{sidebar|UKF0.pdf calls out this matrix only in the text and is inconsistent with the units. The units of variance must be $\mbox{m}^2$ and $\mbox{mrad}^2$ but are specified as $\mbox{m}$ and $\mbox{mrad}$. To correct this, either change the units and interpret the numbers as variances, or interpret the numbers as standard deviations and square them for the matrix. The matrix to the right does the former.}}
The measurements have a covariance as follows:

$$\M{R}=\begin{bmatrix}0.001 & 0 \\
0 & 0.017\end{bmatrix}$$

This means that the noise components are uncorrelated.


In [ ]:
x_r=R_0
y_r=0
w_1=sympy.symbols('w_1')
w_2=sympy.symbols('w_2')
r_r=sympy.sqrt((x_1-x_r)**2+(x_2-y_r)**2)+w_1
theta=sympy.atan((x_2-y_r)/(x_1-x_r))+w_2
display(r_r)
display(theta)

The problem initial conditions are:

In [ ]:
t0=0
t1=20
dt=0.1
ts=np.arange(t0,t1,dt) #Time series
n_t=ts.size
x0=np.array([[6500.4],[349.14],[-1.8093],[-6.7967],[0.6932]])
P0=np.diag([1e-6,1e-6,1e-6,1e-6,1e-12])
nstep=2

In [ ]:
#Monte Carlo initial condition
n_cloud=1000 #Number of elements in the point cloud
nr.seed(3217)
xm = pnoise(n_cloud, P0, x0)
#EKF initial condition
xe = x0.copy()
Pe = P0.copy()
#Unscented initial condition
(xu, W) = sigma(x0, P0)

plt.figure("Entry problem")
plt.plot([0], [0], 'kx')

dt=ts[1]-ts[0]
for i, t in enumerate(ts):
    if i % 20 == 0 or i+1==len(ts):
        if i+1==len(ts):
            plt.plot(xm[0, :], xm[1, :], 'r+',label="Monte Carlo point cloud")
        (xmbar, Pm) = pcalc(xm)
        (cm, sm) = pell(3, Pm[0:2,0:2], xmbar[0:2])
        plt.plot(cm, sm, 'r-',label='Monte Carlo' if i==0 else None)

        if i+1==len(ts):
            plt.plot(xe[0, :], xe[1, :], 'bo',label="EKF estimate")
        (ce, se) = pell(3, Pe[0:2,0:2], xe[0:2])
        plt.plot(ce, se, 'b-',label='Linearized' if i==0 else None)

        if i+1==len(ts):
            plt.plot(xu[0, :], xu[1, :], 'g+',label="Sigma point cloud")
        (xubar, Pu) = sigmainv(xu, W)
        (cu, su) = pell(3, Pu[0:2,0:2], xubar[0:2])
        plt.plot(cu, su, 'g-',label='Unscented' if i==0 else None)
    xm = euler(Fre, dt, xm, nstep=nstep)
    xu = euler(Fre, dt, xu, nstep=nstep)
    (xe, Pe) = ekf_step(xe, Pe, dt, Fre, Phire, nstep=nstep)
plt.axis('equal')
plt.legend()
plt.show()